<a href="https://colab.research.google.com/github/RiyanMak/AIM25/blob/Landmark-detection/NueroCAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Setup and Imports
!pip install mediapipe opencv-python matplotlib

import cv2
import mediapipe as mp
import numpy as np
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
from google.colab import files
import io
import time
from IPython.display import display, clear_output, HTML
import pandas as pd

# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles


# Initialize Face Detection
mp_face_detection = mp.solutions.face_detection

# Define visualization settings
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1, color=(0, 255, 0))

# Helper Functions
def detect_faces(image, face_detection):
    """
    Detect faces in an image using MediaPipe Face Detection

    Args:
        image: Image in RGB format
        face_detection: MediaPipe face detection instance

    Returns:
        List of detected face locations
    """
    # Convert to RGB if necessary
    if len(image.shape) == 3 and image.shape[2] == 3:
        rgb_image = image
    else:
        rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Process image to detect faces
    results = face_detection.process(rgb_image)

    faces = []
    if results.detections:
        for detection in results.detections:
            # Get bounding box
            bboxC = detection.location_data.relative_bounding_box
            ih, iw, _ = image.shape
            x, y, w, h = int(bboxC.xmin * iw), int(bboxC.ymin * ih), \
                         int(bboxC.width * iw), int(bboxC.height * ih)
            faces.append((x, y, w, h))

    return faces


def extract_facial_landmarks(image, face_mesh):
    """
    Extract facial landmarks from an image using MediaPipe Face Mesh

    Args:
        image: Image in RGB format
        face_mesh: MediaPipe face mesh instance

    Returns:
        Dictionary mapping face indexes to NumPy arrays of landmark coordinates,
        each of shape (num_landmarks, 3) where the columns represent (x, y, z)
    """
    # Convert to RGB if necessary
    if len(image.shape) == 3 and image.shape[2] == 3:
        rgb_image = image
    else:
        rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Process image to extract landmarks
    results = face_mesh.process(rgb_image)

    faces_landmarks = {}
    if results.multi_face_landmarks:
        for idx, face_landmarks in enumerate(results.multi_face_landmarks):
            landmarks = []
            for landmark in face_landmarks.landmark:
                # Convert normalized coordinates to pixel coordinates
                h, w, _ = image.shape
                x = int(landmark.x * w)
                y = int(landmark.y * h)
                z = landmark.z  # Retain z as float
                landmarks.append([x, y, z])
            # Convert list to a NumPy array
            landmarks_array = np.array(landmarks)
            faces_landmarks[idx] = landmarks_array

    return faces_landmarks


def visualize_landmarks(image, faces_landmarks, connections=None):
    """
    Draw facial landmarks on an image

    Args:
        image: Image to draw on
        faces_landmarks: Dictionary of face indexes with landmark coordinates
        connections: Optional list of landmark connections to draw

    Returns:
        Image with landmarks drawn
    """
    # Make a copy to avoid modifying the original image
    vis_image = image.copy()

    for face_idx, landmarks in faces_landmarks.items():
        # Draw each landmark as a circle
        for i, (x, y, _) in enumerate(landmarks):
            cv2.circle(vis_image, (x, y), 1, (0, 255, 0), -1)

        # If connections are provided, draw lines between landmarks
        if connections:
            for connection in connections:
                start_idx, end_idx = connection
                if start_idx < len(landmarks) and end_idx < len(landmarks):
                    start_point = landmarks[start_idx][:2]  # (x, y) only
                    end_point = landmarks[end_idx][:2]  # (x, y) only
                    cv2.line(vis_image, start_point, end_point, (255, 0, 0), 1)

    return vis_image

def visualize_face_mesh(image, multi_face_landmarks):
    """
    Draw face mesh on an image using MediaPipe's drawing utilities

    Args:
        image: Image to draw on
        multi_face_landmarks: MediaPipe face landmarks

    Returns:
        Image with face mesh drawn
    """
    # Make a copy to avoid modifying the original image
    vis_image = image.copy()

    # Draw the face mesh on the image
    for face_landmarks in multi_face_landmarks:
        mp_drawing.draw_landmarks(
            image=vis_image,
            landmark_list=face_landmarks,
            connections=mp_face_mesh.FACEMESH_TESSELATION,
            landmark_drawing_spec=None,
            connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_tesselation_style())

        # Draw the face contours
        mp_drawing.draw_landmarks(
            image=vis_image,
            landmark_list=face_landmarks,
            connections=mp_face_mesh.FACEMESH_CONTOURS,
            landmark_drawing_spec=None,
            connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_contours_style())

    return vis_image



# Function to extract specific facial features (for Week 3)
def extract_facial_features(landmarks):
    """
    Extract key facial features from landmarks that may be relevant for neurological disorder detection
    This is a placeholder for Week 3's feature extraction task

    Args:
        landmarks: List of landmark coordinates (x, y, z)

    Returns:
        Dictionary of extracted features
    """
    # Eye aspect ratio (measure of eye openness)
    # Left eye indices (simplified)
    left_eye_top = landmarks[159]
    left_eye_bottom = landmarks[145]
    left_eye_left = landmarks[33]
    left_eye_right = landmarks[133]

    # Right eye indices (simplified)
    right_eye_top = landmarks[386]
    right_eye_bottom = landmarks[374]
    right_eye_left = landmarks[362]
    right_eye_right = landmarks[263]

    # Mouth indices (simplified)
    mouth_left = landmarks[61]
    mouth_right = landmarks[291]
    mouth_top = landmarks[0]
    mouth_bottom = landmarks[17]

    # Calculate eye aspect ratios
    def calculate_distance(point1, point2):
        return np.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)

    left_eye_height = calculate_distance(left_eye_top, left_eye_bottom)
    left_eye_width = calculate_distance(left_eye_left, left_eye_right)
    left_eye_ratio = left_eye_height / left_eye_width if left_eye_width > 0 else 0

    right_eye_height = calculate_distance(right_eye_top, right_eye_bottom)
    right_eye_width = calculate_distance(right_eye_left, right_eye_right)
    right_eye_ratio = right_eye_height / right_eye_width if right_eye_width > 0 else 0

    # Mouth aspect ratio
    mouth_height = calculate_distance(mouth_top, mouth_bottom)
    mouth_width = calculate_distance(mouth_left, mouth_right)
    mouth_ratio = mouth_height / mouth_width if mouth_width > 0 else 0

    # Return extracted features
    features = {
        'left_eye_ratio': left_eye_ratio,
        'right_eye_ratio': right_eye_ratio,
        'eye_symmetry': abs(left_eye_ratio - right_eye_ratio),
        'mouth_ratio': mouth_ratio
    }

    return features




def main():
    # Load test image
    image_path = "test_image.jpg"  # Change to your test image path
    image = cv2.imread(image_path)
    if image is None:
        print("Error: Image not found.")
        return

    # Initialize MediaPipe models
    face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, refine_landmarks=True)
    face_detection = mp_face_detection.FaceDetection(min_detection_confidence=0.5)

    # Detect faces
    faces = detect_faces(image, face_detection)
    print(f"Detected {len(faces)} face(s).")

    # Extract landmarks
    landmarks = extract_facial_landmarks(image, face_mesh)
    if not landmarks:
        print("No facial landmarks detected.")
        return

    # Print landmark shape for verification
    for idx, lm in landmarks.items():
        print(f"Face {idx}: Landmarks shape {lm.shape}")

    # Visualize landmarks
    vis_image = visualize_landmarks(image, landmarks)
    cv2.imshow("Landmarks", vis_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


# Main Execution
if __name__ == "__main__":
    main()
